# Crawling & Normalisasi Data Web PTA

# 1. Library Yang Dibutuhkan

In [2]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.1 MB/s eta 0:00:00


In [3]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import requests
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
# tokenizer = RegexpTokenizer(r'\w+')
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import OneHotEncoder

# 2. Crawling Tugas Akhir Web PTA

Crawling data atau web crawling adalah proses ekstraksi informasi dari situs web dengan menggunakan program komputer. Tujuan utama dari web crawling adalah untuk mengumpulkan data dari berbagai sumber di internet.
Terdapat berbagai macam library yang dapat digunakan untuk crawling data. Library yang digunakan untuk crawling data Web PTA yaitu Beautiful Soup dan Requests. Beautiful Soup Digunakan untuk mengekstrak informasi dari HTML dan Request digunakan untuk mengirim permintaan HTTP dan mengakses halaman web.

In [ ]:
#Dictionary untuk menyimpan data hasil crawling
csv = {"Judul": [], "Penulis": [], "Pembimbing I": [], "Pembimbing II": [], "Abstrak": []}

#Perulangan untuk mengakses web dari halaman 1 sampai halaman 116
for i in range (1,117):
    url = "https://pta.trunojoyo.ac.id/c_search/byprod/4/{}".format(i)
    client = requests.get(url)
    page_html = client.content
    page_soup = soup(page_html, "html.parser")
    tugas_akhir = page_soup.findAll("li", {"data-id":"id-1", "data-cat":"#luxury"})

    #Perulangan untuk mengambil data yang terdapat di web dari tugas akhir teratas sampai tugas akhir terbawah
    for h in tugas_akhir:
      r = requests.get(h.select_one('a.gray.button')['href'])
      page = soup(r.content, "html.parser")
      abstrak_isi = page.select_one('div#content_journal')

      #Mengambil data judul tugas akhir
      judul_tugas = h.select("a", {"class":"title"})
      judul = judul_tugas[0].text

      #Mengambil data nama penulis tugas akhir
      penulis_tugas = h.div.select("span")
      penulis = penulis_tugas[0].text

      #Mengambil data nama pembimbing I
      pembimbing1 = h.div.select("span")
      p1 = penulis_tugas[1].text

      #Mengambil data nama pembimbing II
      pembimbing2 = h.div.select("span")
      p2 = penulis_tugas[2].text

      #Mengambil data abstrak tugas akhir
      abstraks = abstrak_isi.select("p", {"align":"justify"})
      abstrak = abstraks[0].text

      #Menambahkan data yang sudah diambil ke dalam list
      csv["Judul"].append(judul)
      csv["Penulis"].append(penulis)
      csv["Pembimbing I"].append(p1)
      csv["Pembimbing II"].append(p2)
      csv["Abstrak"].append(abstrak)

#Menyimpan data yang ada pada dictionary ke dalam file csv
      data = pd.DataFrame(csv)
      data.to_csv("data_pta.csv",index=False)

# 3. Normalisasi Data

# 3.1. Import Data


Data yang digunakan merupakan data tugas akhir dari web PTA yang sudah dicrawling.

In [4]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
csv_path = 'https://raw.githubusercontent.com/ach-Ilhamf/data_csv/main/data_pta.csv'
data = pd.read_csv(csv_path)
data

,Judul,Penulis,Pembimbing I,Pembimbing II,Abstrak
0,"Efektifitas Pemberian Nutrisi Air Leri, Air Ke...",Penulis : Yusriansyah,Dosen Pembimbing I : Dr. Ir. H. Ahmad Arsyadmu...,"Dosen Pembimbing II :Ir. Sinar Suryawati, MSi",Yusriansyah. 08.03.1.1.1.00005. Efektifitas Pe...
1,RESPON PERTUMBUHAN VEGETATIF TANAMAN HIAS ILER...,Penulis : Nurhayati,"Dosen Pembimbing I : Mustika Tripatmasari, SP....","Dosen Pembimbing II :Catur Wasonowati, SP.,MSi",ABSTRAK\r\n\r\nIler (Coleus blumei) merupakan ...
2,Pengaruh Media Tanam Dan Nutrisi Terhadap Pert...,Penulis : Balia Perwitasari,"Dosen Pembimbing I : Mustika Tripatmasari, SP....","Dosen Pembimbing II :Catur Wasonowati, SP., M.Si.",Pakchoi (Brassica juncea L.) merupakan salah s...
3,UJI PUPUK SULFUR TERHADAP PERTUMBUHAN DAN HASI...,Penulis : Adhadiyanto,"Dosen Pembimbing I : Dr. Ir. RA Sidqi Zaed ZM, MS","Dosen Pembimbing II :Dr. Ir. Eko Murniyanto, MP",Bawang merah merupakan salah satu komoditas un...
4,KERAGAMAN KANDUNGAN MINYAK ATSIRI BUAH CABE JA...,Penulis : Refkie Ardiansyah,Dosen Pembimbing I : Ir. Sinar Suryawati M.Si,Dosen Pembimbing II :Drs.Kaswan Badami M.Si,Cabe jamu (Piper retrofractum Vahl) merupakan ...
...,...,...,...,...,...
572,"PENGARUH DOSIS PUPUK N, P, K TERHADAP PERTUMBU...",Penulis : ASIYATUL BADRIYAH,"Dosen Pembimbing I : Ir. AMIN ZUCHRI, MP",Dosen Pembimbing II :Dr. Ir. H. AHMAD ARSYAD M...,Permasalahan yang sering ditemui yaitu produks...
573,Respon Pertumbuhan dan Hasil Tanaman Kolesom (...,Penulis : Merliana Dewi Astutik,"Dosen Pembimbing I : Ir. Sinar Suryawati, M.Si","Dosen Pembimbing II :Dr. Agr. Eko Setiawan, S....",ABSTRAK\n\nTanaman Kolesom (Talinum triangular...
574,Respon Tanaman Kolesom (Talinum triangulare (J...,Penulis : Yeni Novitasari,"Dosen Pembimbing I : Ir. Sinar Suryawati, M.Si","Dosen Pembimbing II :Dr. Ir. Eko Murnianto, MP",Tanaman kolesom (Talinum. triangulare (Jacq) W...
575,Pengaruh Pemberian Pupuk Kandang Sapi Terhadap...,Penulis : Mar'atus Sholihah,"Dosen Pembimbing I : Ir. Sinar Suryawati,M.Si","Dosen Pembimbing II :Dr. Agr. Eko Setiawan, S....",Penelitian ini bertujuan mengetahui pengaruh p...


In [ ]:
data=data.astype(str)
data["Abstrak"] = data["Abstrak"].apply(lambda x: x.lower())

abstrak = data["Abstrak"]

# 3.2. Tokenisasi

Tokenisasi adalah proses mengonversi teks atau data menjadi serangkaian unit diskrit yang disebut "token." Token adalah unit dasar yang terdiri dari satu atau beberapa karakter, dan dapat berupa kata, frasa, atau karakter tergantung pada tingkat tokenisasi yang diterapkan. Tokenisasi dapat menggunakan library nltk dengan modul word_tokenize.

In [ ]:
def tokenisasi (teks):
  tokens = word_tokenize(teks)
  return tokens

normalisasi = abstrak.apply(tokenisasi)
data['normalisasi_abstrak'] = normalisasi

df = pd.DataFrame(data, columns=['normalisasi_abstrak'])
df

,normalisasi_abstrak
0,"[yusriansyah, ., 08.03.1.1.1.00005., efektifit..."
1,"[abstrak, iler, (, coleus, blumei, ), merupaka..."
2,"[pakchoi, (, brassica, juncea, l., ), merupaka..."
3,"[bawang, merah, merupakan, salah, satu, komodi..."
4,"[cabe, jamu, (, piper, retrofractum, vahl, ), ..."
...,...
572,"[permasalahan, yang, sering, ditemui, yaitu, p..."
573,"[abstrak, tanaman, kolesom, (, talinum, triang..."
574,"[tanaman, kolesom, (, talinum, ., triangulare,..."
575,"[penelitian, ini, bertujuan, mengetahui, penga..."


# 3.3. Punctuation Process

Proses mengahapus angka dari data, menghilangkan karakter non-alphanumerik kecuali spasi dan titik. Proses menghapus angka dan menghilangkan karakter dapat menggunakan library re (Regular Expressions). Library Re digunakan untuk melakukan operasi penggantian teks menggunakan ekspresi reguler.

In [ ]:
def punctuation_process(tokens):
    cleaned_tokens = [re.sub(r'[.,():-]', '', token) for token in tokens]
    cleaned_tokens = [re.sub(r'\d+', '', token) for token in cleaned_tokens]

    return cleaned_tokens

df['normalisasi_abstrak'] = df['normalisasi_abstrak'].apply(punctuation_process)

# Menampilkan DataFrame dengan kolom baru "processed_abstrak"

data = pd.DataFrame(df, columns=['normalisasi_abstrak'])
data

,normalisasi_abstrak
0,"[yusriansyah, , , efektifitas, pemberian, nutr..."
1,"[abstrak, iler, , coleus, blumei, , merupakan,..."
2,"[pakchoi, , brassica, juncea, l, , merupakan, ..."
3,"[bawang, merah, merupakan, salah, satu, komodi..."
4,"[cabe, jamu, , piper, retrofractum, vahl, , me..."
...,...
572,"[permasalahan, yang, sering, ditemui, yaitu, p..."
573,"[abstrak, tanaman, kolesom, , talinum, triangu..."
574,"[tanaman, kolesom, , talinum, , triangulare, ,..."
575,"[penelitian, ini, bertujuan, mengetahui, penga..."


# 3.4. Stopwords

Stopwords adalah kata-kata umum yang biasanya diabaikan atau dihapus selama proses analisis teks. Kata-kata ini sering muncul dalam bahasa tertentu dan tidak memberikan kontribusi signifikan terhadap analisis suatu teks. Oleh karena itu, penghapusan stopwords dapat membantu meningkatkan efisiensi dan fokus analisis. Proses stopwords dapat menggunakan library nltk. Library nltk menyediakan berbagai kamus stopwords untuk beberapa bahasa, termasuk bahasa Indonesia.

In [ ]:
def stopword(text):
    stop_words = set(stopwords.words('indonesian'))
    custom_stop_words = {'aas', 'aatau', 'ab', 'atau', 'yang', 'dan', 'ini', 'itu', 'tentu', 'pasti', 'tidak', 'mungkin', 'bisa','zk', 'zm', 'zn', 'zpt'}
    stop_words.update(custom_stop_words)
    filtered_words = [word for word in text if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['normalisasi_abstrak'] = df['normalisasi_abstrak'].apply(stopword)
# Menampilkan DataFrame dengan kolom baru "processed_abstrak"

data = pd.DataFrame(df, columns=['normalisasi_abstrak'])
data

,normalisasi_abstrak
0,yusriansyah efektifitas pemberian nutrisi ai...
1,abstrak iler coleus blumei salah jenis tanam...
2,pakchoi brassica juncea l salah tanaman sayu...
3,bawang merah salah komoditas unggulan sayuran ...
4,cabe jamu piper retrofractum vahl tanaman pe...
...,...
572,permasalahan ditemui produksi jagung dilahan p...
573,abstrak tanaman kolesom talinum triangulare ...
574,tanaman kolesom talinum triangulare jacq w...
575,penelitian bertujuan pengaruh pemberian dosis ...


# 3.5. Stemming

Stemming adalah proses dalam pemrosesan bahasa alami di mana kata-kata yang berasal dari akar kata yang sama diubah menjadi bentuk dasarnya atau "kata dasar." Tujuan stemming adalah untuk menghapus sufiks atau awalan kata sehingga kata-kata dengan akar yang sama dapat diwakili dengan bentuk yang seragam. Proses stemming dapat menggunakan library Sastrawi.

In [ ]:
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(text)


df['normalisasi_abstrak'] = df['normalisasi_abstrak'].apply(stemming)
# Menampilkan DataFrame dengan kolom baru "processed_abstrak"

data = pd.DataFrame(df, columns=['normalisasi_abstrak'])
data

KeyboardInterrupt: ignored

# 4. Term Frequency (Local Weighting)

In [ ]:
# Membentuk VSM dengan Term Frequency (TF)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['normalisasi_abstrak'])

# Hasil VSM dalam DataFrame
td_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# matriks TF
df = pd.concat([data, td_matrix], axis = 1)
df.to_csv("data_pta2.csv", index = False)
td_matrix

,abad,abariqah,abelmoschus,abelmoscus,abiotik,absolut,absorbsi,absorption,abstract,abstrak,...,zell,zeolit,zibethinus,zinc,zingiber,zingiberaceae,zingiberceae,znso,zona,zuchri
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
573,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Menghitung logaritma dari matriks VSM
log_td_matrix = np.log1p(td_matrix)  # Menggunakan np.log1p untuk menghindari masalah dengan log(0)

log_td_matrix

,abad,abariqah,abelmoschus,abelmoscus,abiotik,absolut,absorbsi,absorption,abstract,abstrak,...,zell,zeolit,zibethinus,zinc,zingiber,zingiberaceae,zingiberceae,znso,zona,zuchri
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.693147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.693147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.693147,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.693147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Membuat CountVectorizer dengan binary
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(data['normalisasi_abstrak'])

# Hasil VSM biner dalam DataFrame
binary_td_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
binary_td_matrix

,abad,abariqah,abelmoschus,abelmoscus,abiotik,absolut,absorbsi,absorption,abstract,abstrak,...,zell,zeolit,zibethinus,zinc,zingiber,zingiberaceae,zingiberceae,znso,zona,zuchri
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
573,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Feature Extraction (TF-IDF)

In [ ]:
# Membuat objek TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF dari data teks
tfidf_matrix = vectorizer.fit_transform(data['normalisasi_abstrak'])

# Membuat DataFrame dari matriks TF-IDF
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# Simpan DataFrame tfidf_df ke dalam file CSV
tfidf_df.to_csv('tfidf_matrix.csv', index=False)
tfidf_df


,abad,abariqah,abelmoschus,abelmoscus,abiotik,absolut,absorbsi,absorption,abstract,abstrak,...,zell,zeolit,zibethinus,zinc,zingiber,zingiberaceae,zingiberceae,znso,zona,zuchri
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
573,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Word2Vec

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

csv_path = '/content/drive/My Drive/Penweb/normalisasi_abstrak.csv'
df = pd.read_csv(csv_path)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,normalisasi_abstrak
0,yusriansyah efektifitas beri nutrisi air leri ...
1,abstrak iler coleus blumei salah jenis tanam h...
2,pakchoi brassica juncea l salah tanam sayur mi...
3,bawang merah salah komoditas unggul sayur penu...
4,cabe jamu piper retrofractum vahl tanam hasil ...
...,...
572,masalah temu produksi jagung lah tani camat so...
573,abstrak tanam kolesom talinum triangulare jacq...
574,tanam kolesom talinum triangulare jacq willd m...
575,teliti tuju pengaruh beri dosis pupuk kandang ...


In [ ]:
df=df.astype(str)

# List token-token dari dokumen
sentences = [token_list.split() for token_list in df['normalisasi_abstrak']]

# Membuat dan melatih model Word2Vec
model = Word2Vec(
    sentences=sentences,
    vector_size=50,
    window=5,
    min_count=1,
    sg=0,
    epochs=10
)

# Melatih model
model.train(sentences, total_examples=len(sentences), epochs=model.epochs)

(581533, 739860)

In [ ]:
# Mengakses vektor kata
try:
    vector = model.wv['nutrisi']
    print(vector)
except KeyError:
    print("Kata tidak ada dalam model.")

[-0.4017986  -0.9952094  -0.46087864  1.4732517  -0.18937805  0.13440625
 -0.41682914 -0.06804504 -1.1476185   1.4110043  -0.13073783  0.03471348
  0.18738815 -0.8510906  -1.1381843   0.07021593  0.75574666 -0.30086574
 -0.37329888 -0.14807051  0.6017379   1.2220986  -0.8131696   0.30275413
  0.8490657  -0.132464    1.2524067  -0.6238855  -0.25037026 -0.18000595
 -0.10643766 -0.34738177 -0.08099767 -0.5835612  -1.442352    1.2327363
  0.42520332 -0.5811758  -0.8753914  -0.19067521  0.25713775 -1.0945385
  0.7678203   1.3935026   0.9732004   1.1987506  -1.2793901   0.3909576
 -1.0867282  -0.06322975]


#LDA

In [ ]:
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords, strip_punctuation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

csv_path = '/content/drive/My Drive/Penweb/normalisasi_abstrak.csv'
df = pd.read_csv(csv_path)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,normalisasi_abstrak
0,yusriansyah efektifitas beri nutrisi air leri ...
1,abstrak iler coleus blumei salah jenis tanam h...
2,pakchoi brassica juncea l salah tanam sayur mi...
3,bawang merah salah komoditas unggul sayur penu...
4,cabe jamu piper retrofractum vahl tanam hasil ...
...,...
572,masalah temu produksi jagung lah tani camat so...
573,abstrak tanam kolesom talinum triangulare jacq...
574,tanam kolesom talinum triangulare jacq willd m...
575,teliti tuju pengaruh beri dosis pupuk kandang ...


In [ ]:
# Mengubah menjadi list token-token
documents = data['normalisasi_abstrak'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Membuat representasi teks dalam bentuk "bag of words"
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Membuat dan melatih model LDA
lda_model = LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)

# Menghitung proporsi kata dalam setiap topik
proporsi_kata = lda_model.get_topics()

# Membuat DataFrame untuk menyimpan proporsi kata dalam topik
proporsi_kata_dalam_topik = pd.DataFrame(proporsi_kata, columns=[dictionary[i] for i in range(len(dictionary))])

proporsi_kata_dalam_topik.to_csv("proporsi_kata_dalam_topik.csv", index=False)
proporsi_kata_dalam_topik


,abstrak,acak,agro,ahmad,air,arsyadmunir,b,bangkalan,banyuajuh,berkhasiat,...,bandung,ditemui,dsn,gabungan,gr/tanaman,uf,dipaangkas,indikator,kedele,konversi
0,0.001923,0.001249,0.000191,0.000116,0.006284,0.000092,0.000399,0.004104,0.000155,0.000142,...,0.000023,0.000023,0.000023,0.000023,0.000023,0.000023,0.000023,0.000023,0.000023,0.000023
1,0.001201,0.004278,0.000014,0.000014,0.003281,0.000014,0.001972,0.002084,0.000016,0.000457,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014
2,0.002142,0.006799,0.000094,0.000072,0.004500,0.000008,0.002499,0.002300,0.000081,0.000292,...,0.000032,0.000032,0.000032,0.000032,0.000297,0.000201,0.000032,0.000032,0.000032,0.000032


In [ ]:
# Menghitung proporsi topik dalam dokumen
proporsi_topik = [lda_model.get_document_topics(doc) for doc in corpus]

# Membuat DataFrame untuk menyimpan proporsi topik dalam dokumen
proporsi_topik_dalam_dokumen = pd.DataFrame(columns=["Dokumen"] + [f"Topik_{i+1}" for i in range(lda_model.num_topics)])

for i, doc_topic_proposals in enumerate(proporsi_topik):
    row_data = {"Dokumen": f"Dokumen {i + 1}"}
    for topic, prop in doc_topic_proposals:
        row_data[f"Topik_{topic + 1}"] = prop
    proporsi_topik_dalam_dokumen = pd.concat([proporsi_topik_dalam_dokumen, pd.DataFrame([row_data])], ignore_index=True)

# Mengganti NaN menjadi 0 persen dalam DataFrame
proporsi_topik_dalam_dokumen = proporsi_topik_dalam_dokumen.fillna(0)

# Menyimpan DataFrame proporsi topik dalam dokumen ke dalam file CSV
proporsi_topik_dalam_dokumen.to_csv("proporsi_topik_dalam_dokumen.csv", index=False)

# Menampilkan hasil proporsi topik dalam dokumen
proporsi_topik_dalam_dokumen

,Dokumen,Topik_1,Topik_2,Topik_3
0,Dokumen 1,0.898337,0.0,0.100146
1,Dokumen 2,0.000000,0.0,0.995761
2,Dokumen 3,0.000000,0.0,0.993950
3,Dokumen 4,0.267632,0.0,0.730481
4,Dokumen 5,0.995538,0.0,0.000000
...,...,...,...,...
572,Dokumen 573,0.000000,0.0,0.994954
573,Dokumen 574,0.000000,0.0,0.995017
574,Dokumen 575,0.000000,0.0,0.995143
575,Dokumen 576,0.000000,0.0,0.991973
